In [0]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [4]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [5]:
 cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls data/

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [8]:
 df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
mean_price = np.mean( df['prices_amountmin'] )

In [64]:
y_true = df['prices_amountmin']
y_pred = [mean_price] * y_true.shape[0]

mean_absolute_error(y_true, y_pred)

63.968341227870624

In [10]:
y_true = df['prices_amountmin']
y_pred = [np.median(y_true)] * y_true.shape[0]

mean_absolute_error(y_true, y_pred)

58.01120240700219

In [11]:
y_true = df['prices_amountmin']
price_log_mean = np.expm1( np.mean( np.log1p(y_true) ) )
y_pred = [price_log_mean] * y_true.shape[0]

mean_absolute_error(y_true, y_pred)

58.03539547661041

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [17]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [18]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [19]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_designed for', 'feat_size', 'feat_nickel free',
       'feat_material detail-1', 'feat_weather resistant', 'feat_mirrored',
       'feat_lens socket width', 'feat_seller inventory number',
       'feat_age segment', 'feat_clothing category'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] /df.shape[0] * 100

In [21]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  


In [23]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [37]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_location - country',
 'feat_catalog',
 'feat_recommended location',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_fabrication',
 'feat_multi pack indicator',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_origin of components_cat',
 'feat_assembled in country of origin_cat',
 'feat_fits brands_cat',
 'feat_location - country_cat',
 'feat_heel height_cat',
 'feat_inseam_cat',
 'feat_model no._cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_shoe fastening type_cat',
 'feat_waist_cat',
 'feat_lining material_cat',
 'feat_disclosures_cat',
 'feat_case finish_cat',
 'feat_display technology_cat',
 'feat_nike product #_cat',
 'feat_is recyclable_cat',
 'feat_applicable_cat',
 'feat_contains batteries_cat',
 'feat_series name_cat',
 'feat_purpose_cat',
 '

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [46]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.24048679076623, 4.290919441162598)


Weight,Feature
0.2461 ± 0.0081,brand_cat
0.1060 ± 0.0077,feat_material_cat
0.0251 ± 0.0038,feat_gender_cat
0.0237 ± 0.0022,feat_brand_cat
0.0134 ± 0.0006,feat_shape_cat
0.0099 ± 0.0017,feat_metal type_cat
0.0039 ± 0.0019,feat_style_cat
0.0001 ± 0.0000,feat_sport_cat


In [27]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [35]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
infant                 4
toddler                4
boys'                  3
women                  2
men||women             2
women ,�� unisex       2
youth                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [0]:
!git add matrix_one/day5